In [11]:
import numpy as np
from numpy.random import randn
import random

In [12]:
train_data = { #тренировачные данные
    'vegetable': True,
    'not vegetable': False,
    'meat': False,
    'not meat': True,
    'vegies menu': True,
    'fat': False,
    'potato': True,
    'a lot of vegies': True,
    'whole bunch a meat': False,
    'ate carrot': True,
    'bad beef': False,
    'tofu': True,
    'tomatoes': True,
    'ham': False,
    'pepperoni': False,
    'mushroom': True,
    'salad': True,
    'order some pork': True,
    'lamb': False,
    'sausage': True,
    'i want carrot only': True,
    'garlick': True,
    'this is turkey': False,
    'cucumber': True,
    'pumpkin': True,
    'duck': False,
    'chicken': False,
    'salad with cucumber': True,
    'looks like roasted chicken': False,
    'beef steak': False,
    'pumpkin stew': True,
    'carrot and tomatoes': True,
    'vegie soup': True,
    'steamed ham': False,
    'boiled tofu': True,
    'potato mash': True,
    'have you tasted very grilled duck': False,
    'not a meat sandwich': True,
    'potato with mushroom': True,
    'why did you make garlick oil': True,
    'i am ordered pork with sause': False,
    'vegie pasta with mushrooms': True,
    'taste of meat pizza with turkey': False,
    'there is no vegetable in my salad': True,
    'today is the day of garlick bread with tomatoes': True,
    'crisy chicken better with no tomatoes': False,
    'i isnt think boiled broccoli with carrots and oil are ready': True,
    'vegetable soup with cucumber': True,
    'pork ham with grilled lamb': False,
    'we are gonna to eat some meat': False,
}

test_data = { #проверочные данные
  'want to taste some vegetable menu': True,
  'today ate some vegie soup': True,
  'this is only meat in there': False,
  'today we have roasted beef': False,
  'want some garlick bread': True,
  'there is no cucumber in my salad': True,
  'i am think of my carrot stew': True,
  'this is very bad meat': False,
  'why there in no meat': True,
  'think we better order some tofu': True,
  'looks like this isnt vegetable menu': False,
  'my duck stew isnt ready': False,
  'you are gonna taste my broccoli mash': True,
}

In [13]:
class Network: #класс с рекуррентной нейронной сетью
    def __init__(self, input_size, output_size, hidden_size=64):
        # веса
        self.Whh = randn(hidden_size, hidden_size) / 1000
        self.Wxh = randn(hidden_size, input_size) / 1000
        self.Why = randn(output_size, hidden_size) / 1000

        #biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs): #выполнение передачи нейронной сети при помощи входных данных      
        h = np.zeros((self.Whh.shape[0], 1))

        self.last_inputs = inputs
        self.last_hs = { 0: h }
        
        for i, x in enumerate(inputs): #выполнение шагов нейронной сети
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h

        y = self.Why @ h + self.by #подсчет вывода

        return y, h

    def backprop(self, d_y, learn_rate=2e-2): #функция обратного распространения    
        n = len(self.last_inputs)

        #вычисление dL/dWhy и dL/dby
        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y

        #инициализация dL/dWhh, dL/dWxh, и dL/dbh
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)

        #вычисление dL/dh для последнего h
        d_h = self.Why.T @ d_y

        for t in reversed(range(n)):         #обратное распространение во времени
            #среднее значение: dL/dh * (1 - h^2)
            temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)

            #dL/db = dL/dh * (1 - h^2)
            d_bh += temp

            #dL/dWhh = dL/dh * (1 - h^2) * h_{t-1}
            d_Whh += temp @ self.last_hs[t].T

            #dL/dWxh = dL/dh * (1 - h^2) * x
            d_Wxh += temp @ self.last_inputs[t].T

            #далее dL/dh = dL/dh * (1 - h^2) * Whh
            d_h = self.Whh @ temp

        #отсекаем, чтобы предотвратить разрыв градиентов
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
            np.clip(d, -1, 1, out=d)

        #обновляем вес и смещение с использованием градиентного спуска
        self.Whh -= learn_rate * d_Whh
        self.Wxh -= learn_rate * d_Wxh
        self.Why -= learn_rate * d_Why
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by

In [14]:
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')])) #создание словаря
vocab_size = len(vocab)
print('%d unique words found' % vocab_size)

word_to_idx = { w: i for i, w in enumerate(vocab) } #назначаем индекс каждому слову
idx_to_word = { i: w for i, w in enumerate(vocab) }

def createInputs(text):  #возвращает массив унитарных векторов, которые представляют слова в введенной строке текста
    inputs = []
    
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs

def softmax(xs): #функция активации softmax 
    return np.exp(xs) / sum(np.exp(xs))

def processData(data, backprop=True): #возврат потери рекуррентной нейронной сети и точности для данных
    items = list(data.items())
    random.shuffle(items)

    loss = 0
    num_correct = 0

    for x, y in items:
        inputs = createInputs(x)
        target = int(y)

        out, _ = net.forward(inputs) #прямое распределение
        probs = softmax(out)

        loss -= np.log(probs[target]) #вычисление потери / точности
        num_correct += int(np.argmax(probs) == target)

        if backprop: #создание dL/dy
            d_L_d_y = probs
            d_L_d_y[target] -= 1

            net.backprop(d_L_d_y)  #обратное распределение

    return loss / len(data), num_correct / len(data)

87 unique words found


In [15]:
net = Network(vocab_size, 2) #инициализация нейросети

for epoch in range(1000): #обучение
    train_loss, train_acc = processData(train_data)

    if epoch % 100 == 99:
        print('--- Epoch %d' % (epoch + 1))
        print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

        test_loss, test_acc = processData(test_data, backprop=False)
        print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.106 | Accuracy: 0.940
Test:	Loss 0.674 | Accuracy: 0.692
--- Epoch 200
Train:	Loss 0.093 | Accuracy: 0.920
Test:	Loss 0.797 | Accuracy: 0.615
--- Epoch 300
Train:	Loss 0.082 | Accuracy: 0.940
Test:	Loss 1.068 | Accuracy: 0.692
--- Epoch 400
Train:	Loss 0.075 | Accuracy: 0.940
Test:	Loss 1.080 | Accuracy: 0.692
--- Epoch 500
Train:	Loss 0.074 | Accuracy: 0.940
Test:	Loss 0.839 | Accuracy: 0.692
--- Epoch 600
Train:	Loss 0.058 | Accuracy: 0.960
Test:	Loss 0.897 | Accuracy: 0.692
--- Epoch 700
Train:	Loss 0.017 | Accuracy: 1.000
Test:	Loss 1.176 | Accuracy: 0.692
--- Epoch 800
Train:	Loss 0.007 | Accuracy: 1.000
Test:	Loss 1.245 | Accuracy: 0.692
--- Epoch 900
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 1.213 | Accuracy: 0.692
--- Epoch 1000
Train:	Loss 0.003 | Accuracy: 1.000
Test:	Loss 1.175 | Accuracy: 0.692


In [17]:
data = ['this is vegies in soup']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  this is vegies in soup 
About vegatables:  [0.99991136] 
About meat:  [8.86368719e-05] 



In [18]:
data = ['i ordered a lot of meat']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  i ordered a lot of meat 
About vegatables:  [0.97170381] 
About meat:  [0.02829619] 



In [19]:
data = ['i ordered a lot of roasted meat']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  i ordered a lot of roasted meat 
About vegatables:  [0.0570876] 
About meat:  [0.9429124] 



In [20]:
data = ['there is no meat in my pizza']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  there is no meat in my pizza 
About vegatables:  [0.99324166] 
About meat:  [0.00675834] 



In [21]:
data = ['i want garlick oil in my salad']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  i want garlick oil in my salad 
About vegatables:  [0.99999985] 
About meat:  [1.49612297e-07] 



In [22]:
data = ['pumpkin stew with beef and tomatoes']

for i in range (len(data)):
    inputs = createInputs(data[i])
    out, h = net.forward(inputs)
    probs = softmax(out)
    print('Input: ', data[i] , '\nAbout vegatables: ', probs[1], '\nAbout meat: ', probs[0], '\n')

Input:  pumpkin stew with beef and tomatoes 
About vegatables:  [0.99870088] 
About meat:  [0.00129912] 

